# Analysis of House Prices in Ames, IA
### Regularized Regression, Cross-Validation, and Feature Selection
##### Grant Nikseresht, Yuqing Zhao, Yue Ning

This notebook is a glimpse into the R workflow that Yuqing, Yue, and myself (Grant) used in analyzing the [Ames, IA housing dataset](https://www.kaggle.com/c/house-prices-advanced-regression-techniques) featured on Kaggle. Our analysis was done as part of our final project for Applied Stats (MATH 564) at IIT. 

The goal of the Kaggle challenge was to predict the selling price of a home given a number of its attributes. Selecting which features to use in our model was the primary challenge in analyzing this dataset, where many of the features are collinear, trivial, or uncorrelated with the dependent variable. 

For our course project, we decided to compare different methods for feature selection including manual selection, stepwise regression, and regularized regression. In this notebook, we'll explore the dataset, implement several regression methods with cross-validation, and compare some results. 


In [ ]:
library(rpart)
library(caret)
library(leaps)
library(glmnet)
library(ggplot2)
library(tabplot)
library(reshape2)
options(warn=-1) 

## Exploring the Data

We performed some preprocessing of the original Kaggle data and stored it in a file in the `data` folder. Let's load it into a dataframe and pull the index out to prepare it for analysis.

In [ ]:
train_df <- read.csv("./data/train_processed.csv")
train_df <- train_df[,-c(1)]

Our processed dataset now contains 1457 observations and 51 explanatory variables. All missing and nonsensical values have been removed. Let's take a look at the data using some handy R functions.

In [ ]:
dim(train_df)
summary(train_df)

Let's take a quick visual glimpse at the distribution of the log of selling price. We use both the built in R functions like `boxplot` and the more extensive plotting package `ggplot2`. 

In [ ]:
boxplot(train_df$SalePrice)
ggplot(data=train_df, aes(train_df$SalePrice)) + 
  geom_histogram (col="red", aes(fill=..count..)) +
  scale_fill_gradient("Count", low = "green", high = "red")+
  labs(title="Histogram for SalePrice") +
  labs(x="SalePrice", y="Count")+
  theme(plot.title = element_text(hjust = 0.5))

Let's take a 10,000 foot view of the explanatory variables as well using `tabplot`. A few are shown below, but feel free to try this on other variables. This view proves useful for visualizing how homoegenous or incomplete a dataset is.

In [ ]:
tableplot(train_df[,30:34])
tableplot(train_df[,41:45])
tableplot(train_df[,5:9])

Here's an example of an issue in the data that required some preprocessing. Each component of the house generally had a few associated explanatory variables. For instance, there are several variables each providing similar information about basements and garages. The `xtabs` function below shows a contingency table to estimate the amount of collinearity between factor variables. 

In [ ]:
xtabs(~GarageQual+GarageCond+GarageFinish, data=train_df)
xtabs(~BsmtCond+BsmtFinType1, data=train_df)

Perfectly collinear or homogenous variables leading can sabotage regression variables, so we're only going to keep one of the variables for garage and one for basement. 

In [ ]:
train_df <- train_df[, -c(which(colnames(train_df) == "GarageFinish"),
                            which(colnames(train_df) == "Exterior2nd"),
                            which(colnames(train_df) == "GarageCond"),
                            which(colnames(train_df) == "BsmtFinType1"))]
dim(train_df)

Here's a function we'll use at the end to compute our error. We're using root mean squared logarithmic error (RMSLE) to compare models. We've already log transformed our dependent variable, so this is basically just RMSE.

$RMSLE = \sqrt{(\frac{1}{n}\sum_{i=1}^n(\hat{Y} - Y)^2)}$

In [ ]:
rmsle <- function(yhat, y) {
  n <- length(yhat)
  return(sqrt((1/n)*sum((yhat-y)^2)))
}

## Analysis

Since the only labeled data we have access to is the training set, we're going to use cross-validation to estimate how well our model will generalize to new data. 

Cross-validation involves splitting a dataset into $k$ different folds (or partitions) of equal size and performing $k$ iterations of model training. On each iteration, $k-1$ folds are selected to make up the training set with the last fold held out as the test set. The model is trained on the training set and then RMSE is estimated on the held out fold. The cross-validation error is the average of the hold-out errors. The RMSE reported here will be the cross-validated RMSE which is given by,

$$RMSE_{CV} =\frac{1}{k} \sum_{i=1}^{k} \sqrt{MSE_{i}}$$

We're going to use the `caret` package, which provides an interface for cross-validating models on a variety of methods. A control parameter is initialized below that will tell future calls to `caret` what settings to use for performing cross-validation. We used 10 folds in our analysis, but we'll only use 3 here so training will finish in a reasonable time. Feel free to increase the number if you're trying this on your own.

In [ ]:
set.seed(564)
controlParameter <- trainControl(method = "cv",
                                  number = 3,
                                  savePredictions = TRUE)

#### Ordinary Least Squares Regression (OLS)

We start with an OLS model containing all of the explanatory variables to establish a performance baseline. The OLS model is given by $y=\beta_0+x_1\beta_1+x_2\beta_2+\dots+x_k\beta_k + \epsilon$. The objective is to select the $\beta$ coefficients to minimize the total error over the training set.

We use `caret`'s function `train` which is a general purpose training function that works with a variety of models. `train` takes in the control parameters we defined above and performs training in a cross-validated way. 

After training our model, we'll extract the final model which extracts the model with the best performance across the given parameters.

In [ ]:
lm_ols <- train(SalePrice~.,
                data = train_df,
                method='lm',
                trControl=controlParameter)
ols_fit <- lm_ols$finalModel

R has a couple of built-in functions that can produce useful results when applied to a variety of objects. 

The `summary` function will produce an overview of a trained model in an easy to read form. The `plot` function will sometimes generate plots related to a mdoel when you pass it the model object. When I try a new package, I'll often just try applying these functions to see what (if anything) is produced. You can see the output for our `caret` model below.

In [ ]:
summary(ols_fit)

In [ ]:
plot(ols_fit)

The baseline model performance is shown below in terms of RMSE and $R^2$. All later models will be judged by whether
or not they improve on these numbers.

In [ ]:
lm_ols$results

#### Manual Feature Selection OLS

For the next model, we applied some heuristics to reduce the number of explanatory variables to one per 'category'. We take categories to be each of the different components of a house such as the bathroom, garage, or zoning type. We keep only the most significant variable in each general category from the first model. The motivation here was to try to build a model with a smaller amount of multicollinearity using some manual selection.

In [ ]:
lm_cats <- train(SalePrice~TotBathrooms+SaleCondition+GarageArea+
                   KitchenQual+GrLivArea+TotalBsmtSF+OverallCond+OverallQual+
                   BldgType+Condition1+MSZoning,
                 data=train_df, 
                 method="lm",
                 trControl=controlParameter)
cats_fit <- lm_cats$finalModel

In [ ]:
summary(cats_fit)

Our first naive approach was not enough to improve performance and actually decreased model performance quite a bit. In other words, there is quite a bit of useful information contained in some of the collinear variables, so we need a more nuanced method for feature selection.

In [ ]:
lm_cats$results

#### Forward Stepwise Regression

The first automatic method for feature selection we'll try is forward stepwise regression. This method begins with the null model containing no variables and systematically adds one variable at a time. This is carried out, generally, until there are no variables that can be added that improve on some set criterion. Here we use the default criterion, BIC. 

On each iteration, the current model is refit with each of the remaining variables. Amongst all the new models (and the current model), the one with the lowest BIC score is chosen as the model for the next iteration. 

This is our first method fit using `caret`'s parameter tuning feature. Forward selection depends on the maximize size of model available given by the parameter `nvmax`. We try forward selection for model sizes up to 180 variables (recall that factor variables must be turned into binary variables, increasing the actual number of explanatory variables). 

In [ ]:
lm_forward <- train(SalePrice~., 
                    data=train_df,
                    method='leapForward',
                    trControl=controlParameter,
                    tuneGrid = expand.grid(nvmax = seq(1, 180, 1))) # set a grid of parameters to try
fwd_fit <- lm_forward$finalModel

lm_forward$results[lm_forward$bestTune[1,1],]

#### Backward Stepwise Regression

Backward stepwise regression is the inverse of forward stepwise regression - start with the full model and start removing variables. We use the same tuning grid here as with forward stepwise regression, but get different results.

It's worth noting that there are $2^k$ possible models where $k$ is the number of explanatory variables, so trying every possible model and picking the best one is often impossible. In our case with around 180 explanatory variables, there are $1.532 x 10^{54}$ possible models for us to choose from. 

In [ ]:
lm_backward <- train(SalePrice~., 
                     data=train_df,
                     method='leapBackward', 
                     trControl=controlParameter, 
                     tuneGrid = expand.grid(nvmax = seq(1, 180, 1)))
bwd_fit <- lm_backward$finalModel

The `caret` training object provides a great visualization of parameter tuning by default using the `plot` function. Here, we can see how the $RMSE_{CV}$ score changes with the number of predictors.

In [ ]:
plot(lm_backward)

### Regularized Regression

A slight expansion of the linear regression model formulation can help to solve our problem of multicollinearity in the dataset and improve the cross-validation error. Rather than using RMSE on its own, the objective function is expanded to include two penalization terms. This new objective function is below where $\lambda$ and $\alpha$ are parameters to be selected. 

$$\min_{\beta_0, \beta} \frac{1}{N} \sum_{i=1}^N RMSE + \frac{\lambda[(1-\alpha)||{\beta}_2^2||}{2} + \alpha||\beta_1||$$

The effect of altering the objective function amounts to 'shrinking' the $\beta$ coefficients for larger values of $\lambda$. Most importantly, $\beta$ coefficients for insignificant variables are shrunken to zero (or near zero) for insignificant factors. In this sense, regularized regression methods are also useful for feature selection. 

Here we'll compare the three prevailing regularization techniques, which each make different assumptions in the objective function.

#### Ridge Regression

Ridge regression is the case where $\alpha = 0$. We'll use `caret` here to select an optimal value of $\lambda$. We test values between 0.1 and 0.00001.


In [ ]:
lambdas <- 10^seq(-1, -5, length = 100) 
ridgeGrid <- expand.grid(alpha=0,lambda=lambdas)
lm_ridge <- train(SalePrice~., data=train_df, method = 'glmnet', trControl=controlParameter, tuneGrid=ridgeGrid)
ridge_fit <- lm_ridge$finalModel

In [ ]:
lm_ridge$results[as.numeric(rownames(lm_ridge$bestTune)),]

In [ ]:
plot(lm_ridge)

The following plot shows how certain variables weights are shrunk across values of the L1 norm. Variables that are quickly shrunk towards zero are considered less important and 'removed' by the regularization method. Each color in the plot is a different variable, although we have too many variables here to distinguish between them. Nonetheless, it's a good visualization of what's going on in regularization.

In [ ]:
plot(ridge_fit)

#### LASSO Regression

LASSO is the case where $\alpha=1$. We use a similar, but more fine-grained, grid of $\lambda$ values.

In [ ]:
lambdas <- 10^seq(-2, -5, length = 300) 
lassoGrid <- expand.grid(alpha=1,lambda=lambdas)
lm_lasso <- train(SalePrice~., data=train_df, method = 'glmnet', trControl=controlParameter, tuneGrid=lassoGrid)
lasso_fit <- lm_lasso$finalModel

In [ ]:
lm_lasso$results[as.numeric(rownames(lm_lasso$bestTune)),]

In [ ]:
plot(lm_lasso)

In [ ]:
plot(lasso_fit)

#### ElasticNet Regression

ElasticNet regression gives partial weight to both the ridge and LASSO penalties. This amounts to varying $\alpha$ between 0 and 1. In this case, we need to tune both $\alpha$ and $\lambda$, leading to a greater computation time, but hopefully a more fine tuned model.

In [ ]:
elasGrid <- expand.grid(alpha=seq(0, 1, length=21),lambda=lambdas)
lm_elas <- train(SalePrice~., data=train_df, method = 'glmnet', trControl=controlParameter, tuneGrid=elasGrid)
elas_fit <- lm_elas$finalModel

In [ ]:
lm_elas$results[as.numeric(rownames(lm_elas$bestTune)),]

With two parameters to tune, you can see in the following graph how RMSE varies over both of them.

In [ ]:
plot(lm_elas)

#### Tree Regression

One final method we decided to try was tree regression. This is a non-parametric method that searches for an optimal series of splits in explanatory variables to estimate the dependent variable. This class of methods is generally known as decision trees, and it will become clear here that they're not well suited for this problem. 

We optimize trees over a complexity paramter, which constrains the size of final trees. 

In [ ]:
treeGrid <- expand.grid(cp=10^seq(-5,-3, length=101))
tree_cp <- train(SalePrice~.,
                 data=train_df,
                 method='rpart',
                 trControl=controlParameter,
                 tuneGrid=treeGrid)
# Zeroing in on the optimal value
treeFineGrid <- expand.grid(cp=seq(0.0002,.0004, length=101))
tree_cp <- train(SalePrice~.,
                 data=train_df,
                 method='rpart',
                 trControl=controlParameter,
                 tuneGrid=treeFineGrid)
tree_fit <- tree_cp$finalModel

In [ ]:
plot(tree_cp)

In [ ]:
tree_cp$results[as.numeric(rownames(tree_cp$bestTune)),]

## Comparison of Methods

In this section, we'll summarize and compare the results. We're interested in seeing whether or not regularization methods are better tools for feature selection than stepwise regression or OLS. 

First, let's gather predictions, $\hat{y}$ over the entire dataset for each method using the `predict` function.

In [ ]:
lm_ols_pred <- predict(lm_ols,train_df)
lm_cats_pred <- predict(lm_cats,train_df)
lm_forward_pred <- predict(lm_forward,train_df)
lm_backward_pred <- predict(lm_backward,train_df)
lm_lasso_pred <- predict(lm_lasso,train_df)
lm_ridge_pred <- predict(lm_ridge,train_df)
lm_elas_pred <- predict(lm_elas,train_df)
tree_cp_pred <- predict(tree_cp,train_df)

Let's also maintain the residuals for diagnostics later. Patterns in the residuals could clue us in on systematic errors in our models that we may need to correct in future work. Residuals are the difference between our model's prediction and the actual value, $\hat{y} - y$.

In [ ]:
ols_res <- ols_fit$residuals
cats_res <- cats_fit$residuals
fwd_res <- lm_forward_pred - train_df$SalePrice
bwd_res <- lm_backward_pred - train_df$SalePrice
lasso_res <- lm_lasso_pred - train_df$SalePrice
ridge_res <- lm_ridge_pred - train_df$SalePrice
elas_res <- lm_elas_pred - train_df$SalePrice
tree_res <- tree_cp_pred - train_df$SalePrice

Now let's apply our `rmsle` function we wrote earlier by inputting the predictions and actual values of the sale price. 

In [ ]:
lm_rmsle <- rmsle(abs(lm_ols_pred), train_df$SalePrice)
lm_cats_rmsle <- rmsle(abs(lm_cats_pred), train_df$SalePrice)
lm_forward_rmsle <- rmsle(abs(lm_forward_pred), train_df$SalePrice) 
lm_backward_rmsle <- rmsle(abs(lm_backward_pred), train_df$SalePrice)
lm_lasso_rmsle <- rmsle(abs(lm_lasso_pred), train_df$SalePrice)
lm_ridge_rmsle <- rmsle(abs(lm_ridge_pred), train_df$SalePrice)
lm_elas_rmsle <- rmsle(abs(lm_elas_pred), train_df$SalePrice)
tree_cp_rmsle <- rmsle(abs(tree_cp_pred), train_df$SalePrice)

In [ ]:
rmsle_scores <- c(lm_rmsle, lm_cats_rmsle, lm_forward_rmsle,
                  lm_backward_rmsle, lm_ridge_rmsle, lm_lasso_rmsle,
                  lm_elas_rmsle, tree_cp_rmsle)
names(rmsle_scores) <- c("OLS_Full", "OLS_Manual", "OLS_Forward",
                         "OLS_Backward", "Ridge", "LASSO",
                         "Elastic","Tree_CP")
rmsle_scores

These scores come from applying the trained models to the training set. The number we're truly interested in is the generalization error that we estimated with cross-validation. Let's summarize the cross-validation errors below.

In [ ]:
best_lm_ols <- lm_ols$results[as.numeric(rownames(lm_ols$bestTune)),]
best_lm_cats <- lm_cats$results[as.numeric(rownames(lm_cats$bestTune)),]
best_lm_forward <- lm_forward$results[as.numeric(rownames(lm_forward$bestTune)),]
best_lm_backward <- lm_backward$results[as.numeric(rownames(lm_backward$bestTune)),]
best_lm_ridge <- lm_ridge$results[as.numeric(rownames(lm_ridge$bestTune)),]
best_lm_lasso <- lm_lasso$results[as.numeric(rownames(lm_lasso$bestTune)),]
best_lm_elastic <- lm_elas$results[as.numeric(rownames(lm_elas$bestTune)),]
best_tree_cp <- tree_cp$results[as.numeric(rownames(tree_cp$bestTune)),]

In [ ]:
cv_results <- data.frame(method = names(rmsle_scores), 
                         rmse = c(best_lm_ols['RMSE'][1,1],
                                  best_lm_cats['RMSE'][1,1],
                                  best_lm_forward['RMSE'][1,1],
                                  best_lm_backward['RMSE'][1,1],
                                  best_lm_ridge['RMSE'][1,1],
                                  best_lm_lasso['RMSE'][1,1],
                                  best_lm_elastic['RMSE'][1,1],
                                  best_tree_cp['RMSE'][1,1]),
                         rmse_sd = c(best_lm_ols['RMSESD'][1,1],
                                      best_lm_cats['RMSESD'][1,1],
                                      best_lm_forward['RMSESD'][1,1],
                                      best_lm_backward['RMSESD'][1,1],
                                      best_lm_ridge['RMSESD'][1,1],
                                      best_lm_lasso['RMSESD'][1,1],
                                      best_lm_elastic['RMSESD'][1,1],
                                      best_tree_cp['RMSESD'][1,1]))

The plot below shows the cross-validated RMSE for each method along with standard deviation bars. Large bars indicate a big spread in the errors computed at each iteration of cross-validation.

In [ ]:
ggplot(cv_results, aes(x=method, y=rmse)) + 
         geom_dotplot(binaxis = 'y', stackdir = 'center') +
         geom_errorbar(aes(ymin=rmse-rmse_sd, ymax=rmse+rmse_sd), width=.2,
                                  position=position_dodge(.0)) +
         xlab("Method") +
         ylab("Cross-Validation RMSE")

Let's now compile the residuals into a dataframe, so we can plot them to look for patterns. 

In [ ]:
residuals <- data.frame(id = seq(1, length(ols_res)),
                        OLS_Full=ols_res,
                        OLS_Manual=cats_res,
                        OLS_Forward=fwd_res,
                        OLS_Backward=bwd_res,
                        Ridge=ridge_res,
                        LASSO=lasso_res,
                        Elastic=elas_res,
                        Tree=tree_res)
res_melt <- melt(residuals, id.vars = "id")

We saw earlier how to look at some standardized residual plots using R's automatic plotting features on a model. Below, we use `ggplot` to see how residuals varied across observations. This reveals some of the outliers and how different models predicted them.

In [ ]:
ggplot(res_melt, aes(x=id, y=value, color=variable)) + 
  geom_point(alpha=0.3, size=0.75) +
  scale_colour_manual(values=c("red", "blue", "green", "orange",
                               "gray", "brown", "black", "purple")) +
  xlab("Observation Index") +
  ylab("Residual Value") +
  scale_fill_discrete(name = "Model")

Our analysis showed that regularization methods can improve feature selection in regression problems with many collinear variables. Here's a couple of exercises you could try after this workshop to get more acquainted with R:

1. Try increasing the number of folds and seeing what happens. 3 folds is not quite enough to get a robust estimate of cross-validation error. For our analysis, we used 10 folds, but for methods like ElasticNet this can get intensive. 

2. We used a preprocessed dataset here, but I've included the unprocessed dataset in the /data folder in the Git repo. Try starting from that dataset and implementing the same models. See what problems come up and tinker with the dataset.